# **ResNet50**

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision.models as models
from torchvision.models.resnet import ResNet50_Weights
from torchvision import models, transforms
from PIL import Image
import os

In [2]:
train_path = '/kaggle/input/sugarcane-leaf-disease/Sugarcane Leaf Disease/Train'
test_path = '/kaggle/input/sugarcane-leaf-disease/Sugarcane Leaf Disease/Test'

In [3]:
# Define transformations for the images
transformations = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
# Load the datasets
train_dataset = datasets.ImageFolder(root=train_path, transform=transformations)
test_dataset = datasets.ImageFolder(root=test_path, transform=transformations)

In [5]:
# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [6]:
# Load the pretrained ResNet50 model
weights = ResNet50_Weights.IMAGENET1K_V1
model = models.resnet50(weights=weights)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 153MB/s]


In [7]:
# Modify the classifier for the dataset's number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

In [8]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [9]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
# Training the model
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

Epoch 1/15 - Loss: 1.0620, Accuracy: 60.77%
Epoch 2/15 - Loss: 0.2863, Accuracy: 91.58%
Epoch 3/15 - Loss: 0.1241, Accuracy: 96.78%
Epoch 4/15 - Loss: 0.0811, Accuracy: 98.27%
Epoch 5/15 - Loss: 0.0489, Accuracy: 98.96%
Epoch 6/15 - Loss: 0.0466, Accuracy: 99.46%
Epoch 7/15 - Loss: 0.0398, Accuracy: 99.11%
Epoch 8/15 - Loss: 0.0421, Accuracy: 99.65%
Epoch 9/15 - Loss: 0.0258, Accuracy: 99.60%
Epoch 10/15 - Loss: 0.0243, Accuracy: 99.60%
Epoch 11/15 - Loss: 0.0172, Accuracy: 99.50%
Epoch 12/15 - Loss: 0.0187, Accuracy: 99.75%
Epoch 13/15 - Loss: 0.0268, Accuracy: 99.65%
Epoch 14/15 - Loss: 0.0220, Accuracy: 99.46%
Epoch 15/15 - Loss: 0.0551, Accuracy: 99.80%


In [11]:
# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 96.02%


In [12]:
torch.save(model.state_dict(), 'resnet50.pth')

# **Predict Class for Custom Data(Pytorch)**

In [13]:
model = models.resnet50()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))
model.load_state_dict(torch.load('/kaggle/working/resnet50.pth', map_location=device))
model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
# Function to predict the class of a single image
def predict_image(image_path, model, transformations, class_names):
    image = Image.open(image_path).convert('RGB')
    image = transformations(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)

    return class_names[predicted.item()]

In [15]:
class_names = train_dataset.classes
# Predict the class for a custom image
image_path = '/kaggle/input/sugarcane-leaf-disease/Sugarcane Leaf Disease/Train/Yellow/yellow (107).jpeg'
predicted_class = predict_image(image_path, model, transformations, class_names)
print(f"Predicted class: {predicted_class}")

Predicted class: Yellow


In [16]:
# !pip install onnx
# !pip install onnx-tf
# !pip install tensorflow

# **Load Pytorch Model**

In [17]:
# Load the trained PyTorch model
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

In [18]:
# Load the saved weights
model.load_state_dict(torch.load('/kaggle/working/resnet50.pth'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
# Dummy input for the model
dummy_input = torch.randn(1, 3, 224, 224)

# **Export pytorch to ONNX**

In [20]:
# Export the model to ONNX
torch.onnx.export(model, dummy_input, '/kaggle/working/resnet.onnx', input_names=['input'], output_names=['output'], opset_version=11)

# **Export Tensorflow Model**

In [21]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load('/kaggle/working/resnet.onnx')

# Convert the ONNX model to TensorFlow
tf_rep = prepare(onnx_model)

# Export the TensorFlow model
tf_rep.export_graph('/kaggle/working/resnet_tensorflow')

2024-05-18 07:17:27.580099: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 07:17:27.580226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 07:17:27.702705: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our Tens

# **Predict Class for Custom Data(Tensorflow)**

In [22]:
import tensorflow as tf
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

In [23]:
# Load the TensorFlow model
model = tf.saved_model.load('/kaggle/working/resnet_tensorflow')

In [24]:
# Define your class indices
class_indices = {0: 'Healthy', 1: 'Mosaic', 2: 'RedRot', 3: 'Rust', 4: 'Yellow'} # train_dataset.classes

In [25]:
# Function to preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path)
    transformations = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transformations(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image.numpy()

In [26]:
# Path to the custom image
image_path = '/kaggle/input/sugarcane-leaf-disease/Sugarcane Leaf Disease/Train/Yellow/yellow (107).jpeg'
# Preprocess the image
input_tensor = preprocess_image(image_path)

In [27]:
# Run inference
input_tensor = tf.convert_to_tensor(input_tensor, dtype=tf.float32)
output = model.signatures['serving_default'](input_tensor)

In [28]:
# Get the predicted class index
predicted_class_index = np.argmax(output['output'].numpy(), axis=1)[0]
# Get the class name
predicted_class_name = class_indices[predicted_class_index]

In [29]:
# print(f"Predicted class index: {predicted_class_index}")
print(f"Predicted class name: {predicted_class_name}")

Predicted class name: Yellow
